In [ ]:
import sqlite3
import pandas as pd

In [ ]:
dbase = sqlite3.connect("hr.db")

### A. Truy vấn đơn giản

### 1. Liệt kê các nhân viên gồm: mã nhân viên, tên (first_name, last_name), ngày vào làm, lương
(có thể định dạng ngày vào làm và lương)

In [ ]:
sql = '''SELECT employee_id, first_name||' '||last_name AS Name, hire_date, salary FROM employees;'''
df = pd.read_sql_query(sql, dbase)
df.head()

### 2. Liệt kê các nhân viên có thêm cột biểu thức tham_nien (thâm niên = năm hiện tại - năm vào làm)
(có thể sắp giảm theo thâm niên)

In [ ]:
sql = '''SELECT employee_id, first_name, hire_date, strftime('%Y', 'now') - strftime('%Y', hire_date) AS tham_nien
FROM employees
ORDER BY tham_nien DESC;'''
df = pd.read_sql_query(sql, dbase)
df.head()



### 3. Cho biết nhân viên có lương lớn nhất

In [ ]:
sql = ''' SELECT * 	FROM employees
ORDER BY salary DESC
LIMIT 1; ''' 
df = pd.read_sql_query(sql, dbase)
df.head()

### 4. Cho biết nhân viên có lương lớn nhất của phòng 80

In [12]:
sql = ''' SELECT * 
FROM employees
WHERE department_id=80
ORDER BY salary DESC
LIMIT 1; ''' 
df = pd.read_sql_query(sql, dbase)
df.head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
0,145,John,Russell,JRUSSEL,011.44.1344.429268,2004-10-01 00:00:00,SA_MAN,14000,0.4,100,80


### 5. Cho biết nhân viên có lương lớn nhất của mỗi phòng

In [16]:
sql = ''' SELECT e.department_id, e.first_name, e.salary
FROM employees e JOIN 
(SELECT department_id, max(salary) AS max_sal
FROM employees
GROUP BY department_id) g ON e.department_id=g.department_id
WHERE e.salary=g.max_sal
ORDER BY e.department_id; ''' 
df = pd.read_sql_query(sql, dbase)
df.head()

,department_id,first_name,salary
0,10,Jennifer,4400
1,20,Michael,13000
2,30,Den,11000
3,40,Susan,6500
4,50,Adam,8200


### B. Truy vấn có nhóm

### 1. Thống kê theo mã phòng và đếm số nhân viên, tổng lương theo phòng

In [17]:
sql = ''' SELECT department_id, COUNT(*) AS dem, SUM(salary) AS tong
FROM employees
WHERE department_id IS NOT NULL
GROUP BY department_id; ''' 
df = pd.read_sql_query(sql, dbase)
df.head()

,department_id,dem,tong
0,10,1,4400
1,20,2,19000
2,30,6,24900
3,40,1,6500
4,50,45,156400


### 2. Thống kê theo mã phòng, tên phòng và đếm số nhân viên, tổng lương theo phòng

In [18]:
sql = ''' SELECT e.department_id, d.department_name, COUNT(*) AS dem, SUM(salary) AS tong
FROM employees e JOIN departments d ON e.department_id=d.department_id
GROUP BY e.department_id, d.department_name
ORDER BY e.department_id; ''' 
df = pd.read_sql_query(sql, dbase)
df.head()

,department_id,department_name,dem,tong
0,10,Administration,1,4400
1,20,Marketing,2,19000
2,30,Purchasing,6,24900
3,40,Human Resources,1,6500
4,50,Shipping,45,156400


### 3. Cho biết phòng nào có tổng lương lớn nhất

In [19]:
sql = ''' SELECT e.department_id, d.department_name, COUNT(*) AS dem, SUM(salary) AS tong
FROM employees e JOIN departments d ON e.department_id=d.department_id
GROUP BY e.department_id, d.department_name
ORDER BY tong DESC
LIMIT 1; ''' 
df = pd.read_sql_query(sql, dbase)
df.head()

,department_id,department_name,dem,tong
0,80,Sales,34,304500


### C. Truy vấn con

### 1. Tìm các nhân viên có lương > trung bình lương

In [20]:
sql = ''' SELECT *
FROM employees
WHERE salary > (SELECT avg(salary) FROM employees); ''' 
df = pd.read_sql_query(sql, dbase)
df.head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
0,100,Steven,King,SKING,515.123.4567,2003-06-17 00:00:00,AD_PRES,24000,,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21 00:00:00,AD_VP,17000,,100.0,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,2001-01-13 00:00:00,AD_VP,17000,,100.0,90.0
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,2006-01-03 00:00:00,IT_PROG,9000,,102.0,60.0
4,108,Nancy,Greenberg,NGREENBE,515.124.4569,2002-08-17 00:00:00,FI_MGR,12008,,101.0,100.0


### 2. Tìm các nhân viên làm việc tại vị trí phòng là 1700

In [21]:
sql = ''' SELECT *
FROM employees
WHERE department_id IN (SELECT department_id FROM departments WHERE location_id=1700);''' 
df = pd.read_sql_query(sql, dbase)
df.head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
0,100,Steven,King,SKING,515.123.4567,2003-06-17 00:00:00,AD_PRES,24000,,NaN,90
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21 00:00:00,AD_VP,17000,,100.0,90
2,102,Lex,De Haan,LDEHAAN,515.123.4569,2001-01-13 00:00:00,AD_VP,17000,,100.0,90
3,108,Nancy,Greenberg,NGREENBE,515.124.4569,2002-08-17 00:00:00,FI_MGR,12008,,101.0,100
4,109,Daniel,Faviet,DFAVIET,515.124.4169,2002-08-16 00:00:00,FI_ACCOUNT,9000,,108.0,100


### 3. Hiển thị tên trưởng phòng của phòng 80

In [22]:
sql = ''' SELECT *
FROM employees
WHERE employee_id =(SELECT manager_id FROM departments WHERE department_id=80);''' 
df = pd.read_sql_query(sql, dbase)
df.head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
0,145,John,Russell,JRUSSEL,011.44.1344.429268,2004-10-01 00:00:00,SA_MAN,14000,0.4,100,80


### 4. Hiển thị tên trưởng phòng của các phòng 80,90

In [23]:
sql = ''' SELECT *
FROM employees
WHERE employee_id IN(SELECT manager_id FROM departments WHERE department_id IN (80,90));''' 
df = pd.read_sql_query(sql, dbase)
df.head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
0,100,Steven,King,SKING,515.123.4567,2003-06-17 00:00:00,AD_PRES,24000,,NaN,90
1,145,John,Russell,JRUSSEL,011.44.1344.429268,2004-10-01 00:00:00,SA_MAN,14000,0.4,100.0,80
